# Get carbon intensity by country

We get our carbon intensity from [Our World In Data](https://ourworldindata.org/grapher/carbon-intensity-electricity).

>“Data Page: Carbon intensity of electricity generation”, part of the following publication: Hannah Ritchie, Pablo Rosado and Max Roser (2023) - “Energy”. Data adapted from Ember, Energy Institute. Retrieved from https://ourworldindata.org/grapher/carbon-intensity-electricity [online resource]

The data source and processing are available on [OWID Github](https://github.com/owid/energy-data).

We have to:
1. Filter for the last available year
1. Check if we not loose country we previously had.
1. Keep only the data we need
1. Export to JSON

In [14]:
import pandas as pd
import numpy as np
import json

In [15]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv")
df

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
0,ASEAN (Ember),2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
1,ASEAN (Ember),2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
2,ASEAN (Ember),2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
3,ASEAN (Ember),2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
4,ASEAN (Ember),2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21807,Zimbabwe,2019,ZWE,15354606.0,2.514642e+10,NaN,NaN,NaN,NaN,12.374,...,0.353,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21808,Zimbabwe,2020,ZWE,15669663.0,2.317871e+10,NaN,NaN,NaN,NaN,6.382,...,0.298,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21809,Zimbabwe,2021,ZWE,15993525.0,2.514009e+10,NaN,NaN,NaN,NaN,0.000,...,0.236,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN
21810,Zimbabwe,2022,ZWE,16320539.0,2.590159e+10,NaN,NaN,NaN,NaN,6.127,...,0.336,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN


In [16]:
df.query("country == 'India'")

,country,year,iso_code,population,gdp,biofuel_cons_change_pct,biofuel_cons_change_twh,biofuel_cons_per_capita,biofuel_consumption,biofuel_elec_per_capita,...,solar_share_elec,solar_share_energy,wind_cons_change_pct,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_electricity,wind_energy_per_capita,wind_share_elec,wind_share_energy
9205,India,1900,IND,2.807142e+08,2.716975e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9206,India,1901,IND,2.818341e+08,2.773278e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9207,India,1902,IND,2.831998e+08,3.006720e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9208,India,1903,IND,2.848130e+08,3.047644e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9209,India,1904,IND,2.864353e+08,3.060750e+11,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9324,India,2019,IND,1.383112e+09,9.595463e+12,23.248,2.451,9.396,12.996,22.782,...,2.852,1.309,4.979,7.327,166.895,45.774,63.31,120.666,3.903,1.791
9325,India,2020,IND,1.396387e+09,8.945314e+12,-6.260,-0.813,8.724,12.182,23.532,...,3.709,1.746,-4.560,-8.195,158.700,43.276,60.43,113.650,3.820,1.798
9326,India,2021,IND,1.407564e+09,9.801621e+12,61.522,7.495,13.979,19.677,26.116,...,3.984,1.865,12.688,19.481,178.181,48.374,68.09,126.588,3.971,1.859
9327,India,2022,IND,1.417173e+09,1.047625e+13,42.380,8.339,19.769,28.016,28.747,...,5.122,2.456,2.865,4.437,182.617,49.429,70.05,128.860,3.770,1.808


In [17]:
elec_col = [c for c in df.columns.to_list() if "_electricity" in c]
nuke_col = [c for c in elec_col if "nuclear" in c]
nuke_col

['nuclear_electricity']

In [18]:
carbon_intensity_col = [c for c in df.columns.to_list() if "intensity" in c]

In [19]:
latest_year = df['year'].max()
df[['country', 'iso_code', 'year'] + carbon_intensity_col + elec_col].query(f"iso_code=='FRA' and year=={latest_year}")

,country,iso_code,year,carbon_intensity_elec,biofuel_electricity,coal_electricity,fossil_electricity,gas_electricity,hydro_electricity,low_carbon_electricity,nuclear_electricity,oil_electricity,other_renewable_electricity,other_renewable_exc_biofuel_electricity,per_capita_electricity,renewables_electricity,solar_electricity,wind_electricity
7502,France,FRA,2023,56.039,9.5,2.16,43.3,31.43,53.19,470.81,335.65,9.71,10.1,0.6,7939.115,135.16,23.26,48.61


In [20]:
df_filtered = df[['country', 'iso_code', 'year'] + carbon_intensity_col + elec_col].query(f"year=={latest_year} and iso_code.notnull()")
columns_names = {
    'country':'country_name',
    'carbon_intensity_elec':'carbon_intensity',
    'biofuel_electricity':'biofuel_TWh',
    'coal_electricity':'coal_TWh',
    'fossil_electricity':'fossil_TWh',
    'gas_electricity':'gas_TWh',
    'hydro_electricity':'hydroelectricity_TWh',
    'low_carbon_electricity':'low_carbon_TWh',
    'nuclear_electricity':'nuclear_TWh',
    'oil_electricity':'oil_TWh',
    'other_renewable_electricity':'other_renewable_TWh',
    'other_renewable_exc_biofuel_electricity':'other_renewable_exc_biofuel_TWh',
    'per_capita_electricity':'per_capita_Wh',
    'renewables_electricity':'renewables_TWh',
    'solar_electricity':'solar_TWh',
    'wind_electricity':'wind_TWh'
}
df_filtered.rename(columns=columns_names, inplace=True)
df_filtered = df_filtered[['country_name',
                        'iso_code',
                        'year',
                        'carbon_intensity',
                        'fossil_TWh',
                        'renewables_TWh',
                        'per_capita_Wh',
                        'coal_TWh',
                        'gas_TWh',
                        'oil_TWh',
                        'solar_TWh',
                        'wind_TWh',
                        'hydroelectricity_TWh',
                        'other_renewable_TWh',
                        'other_renewable_exc_biofuel_TWh',
                        'nuclear_TWh',
                        'biofuel_TWh',
                        'low_carbon_TWh']]
df_filtered.head(5)

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh
759,Algeria,DZA,2023,NaN,NaN,0.734,1883.719,NaN,NaN,NaN,0.649,0.016,0.069,0.000,NaN,0.00,NaN,0.734
926,Angola,AGO,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN
1135,Argentina,ARG,2023,354.103,91.24,53.240,3351.911,0.60,73.38,17.26,3.250,14.490,33.170,2.276,NaN,8.95,2.33,62.190
1693,Australia,AUS,2023,548.692,175.73,99.160,10397.096,127.52,42.78,5.43,46.910,33.130,15.760,3.126,NaN,0.00,3.36,99.160
1859,Austria,AUT,2023,110.812,10.39,56.570,7474.081,0.06,7.17,3.16,5.160,8.050,39.790,3.570,0.0,0.00,3.57,56.570


In [21]:
columns_to_sum = ["fossil_TWh", "renewables_TWh", "nuclear_TWh"]
df_filtered['total_TWh'] = df_filtered[columns_to_sum].sum(axis=1)
df_filtered.head(2)

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
759,Algeria,DZA,2023,NaN,NaN,0.734,1883.719,NaN,NaN,NaN,0.649,0.016,0.069,0.0,NaN,0.0,NaN,0.734,0.734
926,Angola,AGO,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.000


In [22]:
df_filtered.query("iso_code=='FRA'")

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
7502,France,FRA,2023,56.039,43.3,135.16,7939.115,2.16,31.43,9.71,23.26,48.61,53.19,10.1,0.6,335.65,9.5,470.81,514.11


In [23]:
df_filtered.query("iso_code=='ARE'")

,country_name,iso_code,year,carbon_intensity,fossil_TWh,renewables_TWh,per_capita_Wh,coal_TWh,gas_TWh,oil_TWh,solar_TWh,wind_TWh,hydroelectricity_TWh,other_renewable_TWh,other_renewable_exc_biofuel_TWh,nuclear_TWh,biofuel_TWh,low_carbon_TWh,total_TWh
20244,United Arab Emirates,ARE,2023,NaN,118.933,NaN,17342.172,0.0,118.931,0.003,13.748,0.054,NaN,0.033,NaN,32.274,NaN,NaN,151.207


renewables_Twh = hydroelectricity_Twh + wind_Twh + solar_Twh + other_renewable_Twh

RTE Production

![RTE](2023-07-07-22-40-48.png)


In [24]:
# French electricity consumption in 2022 from https://ourworldindata.org/grapher/elec-mix-bar?stackMode=absolute&facet=metric&country=~FRA
# 57 TWh fossil + 298 TWh nuclear + 116 TWh renewable = 471 TWh
# From https://bilan-electrique-2021.rte-france.com/# we read 522 TWh for Production, but 468 TWh of consumption.
# Here we found 470 TWh, which is pretty close.

df_filtered.query("iso_code=='FRA'")["total_TWh"].values[0]

514.1099999999999

In [25]:
energy_mix = {}
missings = []
for code in df_filtered['iso_code'].unique():
#for code in ['FRA', 'AFG', 'VNM']:
    # print(code)
    df=df_filtered.query("iso_code == @code")
    max_year = df.year.max()
    #if df.isnull().values.any() == False:
    if len(df)>0 and not np.isnan(df.loc[df.year == max_year].iloc[0]["carbon_intensity"]):
        energy_mix[code]=df.loc[df.year == max_year].iloc[0].replace(np.nan, 0).to_dict()
    else:
        missings.append(code)
print(f"Missing carbon_intensity for {len(missings)} countries over {len(df_filtered)}.")

Missing carbon_intensity for 38 countries over 116.


In [26]:
energy_mix["CAN"]

{'country_name': 'Canada',
 'iso_code': 'CAN',
 'year': 2023,
 'carbon_intensity': 170.043,
 'fossil_TWh': 126.1,
 'renewables_TWh': 419.89,
 'per_capita_Wh': 16371.299,
 'coal_TWh': 32.33,
 'gas_TWh': 91.16,
 'oil_TWh': 2.61,
 'solar_TWh': 7.48,
 'wind_TWh': 38.1,
 'hydroelectricity_TWh': 365.39,
 'other_renewable_TWh': 10.384,
 'other_renewable_exc_biofuel_TWh': 0,
 'nuclear_TWh': 88.91,
 'biofuel_TWh': 8.92,
 'low_carbon_TWh': 508.8,
 'total_TWh': 634.9}

In [27]:

np.isnan(df_filtered.loc[df_filtered.iso_code == "CAN"].iloc[0]["carbon_intensity"])

False

In [28]:
with open("global_energy_mix.json", "w") as outfile:
    json.dump(energy_mix, outfile, indent=4, sort_keys=True)

In [29]:
len(energy_mix.keys())
# 81 countries in 2022 and only 78 in 2023, we can do better! ;D

78